# Open Weather API

THis notebook go thru the process to gather the data from an API connection, exported to csv/json for the historical files needed later, create a df for preprocessing, and produce at the end a ready to use data that can be integrated to timeseries_data table of each customer.

_Open Weather_:
Forecast: It produce forecast in a range of 3h for 160h in the future. A new forecast is made in the model every 10'.
Current:

Consideerations:
1. Call for current weather or forecast weather (historical to populate files will be download in one shoot from server)
2. Call 1 site or for multiple sites
3. Automatic call with a given interval
4. Function that connect the specific site(s) lat/lon from the customer and create the df_lat_lon that will be used for call the API
5. Keep a csv historical file to store the progression of the different call
6. All the data stored in 1 file
7. Parameters to be keeped: 
    - Main: Temp, Humidity
    - Secondary: Rain, Sun_exposure, wind_speed, ground_pressure, forecast
8. Preprocess:
    
    - Create the time frame compatible with the 15 minutes interval in the energy consumption interval = 15'
    - Populate the intervals with and linear interpolation of the data from hour x to hour w+new meassure
    - Not Nan data
    - Rain (m3 of rain per hour) NaN converted to 0
    - Keep only the most up to date data (last) records
    - ? Specific df with data only for the site(s) that are pertinent for the customer 
    - ? Store in a dedicated df that will be exported to timeseries_data of the specific project

Author: Camilo Zuluaga

## Libraries

In [1]:
# import libraries
import pandas as pd
import requests
import time
import pathlib
import schedule
import os

ModuleNotFoundError: No module named 'schedule'

## Global Variables

Create connection to data and other sources

In [ ]:
# Variable path_lat_lon with the path to the CSV file (C:\Users\zudel\projects\zapdos\weather_api\data\lat_long_cities.csv) with the cities and their coordinates
path_lat_lon = "C:/Users/zudel/projects/zapdos/weather_api/data/lat_long_cities.csv"

# create csv file to save different if is current or forecast
path_forecast_csv = "weather_data_openweather.csv"
path_forecast_data = f"forecast_data_{path_lat_lon}.csv"

# df_lat_lon with the data of the CSV file
df_lat_lon = pd.read_csv(path_lat_lon)

# Interval time (up_time) for the update of the call
up_time = 15

Variables specific to the API (Openweather)

In [3]:
# API_KEY open weather
API_KEY = "a019bc0712358cb223887c5bf73bb473"


# variable lat, lon Brusselles (testing)
lat = "50.85045"
lon = "4.34878"

In [4]:
# Different URL API call (current, forecast)
# global variable openweather API "current weather"
url_current = (
    "https://api.openweathermap.org/data/2.5/weather?lat="
    + lat
    + "&lon="
    + lon
    + "&exclude=hourly,minutely&appid="
    + API_KEY
    + "&units=metric"
)  # current weather & exlude=current


# global varaiables open weather api **forecast**
url_forecast = (
    "https://api.openweathermap.org/data/2.5/forecast?lat="
    + lat
    + "&lon="
    + lon
    + "&appid="
    + API_KEY
    + "&units=metric"
)  # 5 days forecast 3 hours interval &exclude=current,minutely,hourly

In [ ]:
# # Function to store the data in a CSV file
# def store_data(data, path):
#     # Check if the directory exists, if not, create it
#     directory = os.path.dirname(path)
#     if not os.path.exists(directory):
#         os.makedirs(directory)

#     # Save the data to the file
#     data.to_csv(path, index=False)

## API call

Different functions to call current, forecast

Function to create the API connection and verify if is working. Function intended as a control point to deal with connections issueas and create a log if necessary.

In [5]:
# Function to call the open weather api. Use url_current or url_forecast. Test the connection
def call_open_weather_api(url):
    # Call the API
    response = requests.get(url)
    # Check if the response is correct
    if response.status_code == 200:
        return response.json()
    else:
        return None  # print(f"Error [{response.status_code}]")

In [7]:
# function to call the openweather API and store in a dataframe
def get_forecast():
    # Call the API
    data = call_open_weather_api(url_forecast)
    # Check if the data is correct
    if data:
        df = pd.DataFrame(data["list"])
        return df
    else:
        return None


print(get_forecast())


# function to get the forecast data
def get_forecast_data():
    # Call the API
    df = get_forecast()
    # Check if the data is correct and return the necessary columns
    if df is not None:
        df = df[["dt_txt", "main", "weather", "clouds", "wind", "pop"]]
        return df
    else:
        return None

            dt                                               main  \
0   1715774400  {'temp': 16.32, 'feels_like': 16.15, 'temp_min...   
1   1715785200  {'temp': 16.8, 'feels_like': 16.55, 'temp_min'...   
2   1715796000  {'temp': 16.69, 'feels_like': 16.5, 'temp_min'...   
3   1715806800  {'temp': 13.89, 'feels_like': 13.89, 'temp_min...   
4   1715817600  {'temp': 13.26, 'feels_like': 13.2, 'temp_min'...   
5   1715828400  {'temp': 12.9, 'feels_like': 12.78, 'temp_min'...   
6   1715839200  {'temp': 13.65, 'feels_like': 13.53, 'temp_min...   
7   1715850000  {'temp': 14.62, 'feels_like': 14.46, 'temp_min...   
8   1715860800  {'temp': 14.62, 'feels_like': 14.49, 'temp_min...   
9   1715871600  {'temp': 13.26, 'feels_like': 13.1, 'temp_min'...   
10  1715882400  {'temp': 12.99, 'feels_like': 12.9, 'temp_min'...   
11  1715893200  {'temp': 12.61, 'feels_like': 12.46, 'temp_min...   
12  1715904000  {'temp': 12.21, 'feels_like': 12.02, 'temp_min...   
13  1715914800  {'temp': 12.06, 'f

Function to Call all the forecast data and analize relevance, pertinence, and processing needs

In [ ]:
# funtion to get the forecast data and store it in a dataframe
def get_forecast_data():
    data = call_open_weather_api(url_forecast)
    if data is not None:
        df = pd.json_normalize(data["list"])
        # df = df[['dt','dt_txt','main.temp','main.humidity','wind.speed','clouds.all','main.grnd_level', 'rain.3h']]
        # df.columns = ['dt','date','temperature','humidity','windSpeed','cloudCover','pressure','rain']
        # from weather column keep only 'id'
        df["weather"] = df["weather"].apply(lambda x: x[0]["id"])

        return df

    else:
        return None


print(get_forecast_data().info())

# Loop forseveral points
Function to integrate in the call different sites

In [ ]:
# Function to iterate thru a list of sites and get the forecast data, store it in a dataframe and save it in a csv file:


def get_forecast_data():
    # create df_forecast to store the data
    df_forecast = pd.DataFrame()
    # create df_lat_lon and use data/lat_long_cities.csv (path_lat_lon)
    df_lat_lon = pd.read_csv(path_lat_lon)
    # Create a call with each one of the df_lat_lon rows and store the data in df_forecast
    for index, row in df_lat_lon.iterrows():
        lat = row["Latitude"]
        lon = row["Latitude"]
        url_forecast = (
            "https://api.openweathermap.org/data/2.5/forecast?lat="
            + str(lat)
            + "&lon="
            + str(lon)
            + "&appid="
            + API_KEY
            + "&units=metric"
        )
        data = call_open_weather_api(url_forecast)
        if data is not None:
            df = pd.json_normalize(data["list"])
            df["lat"] = lat
            df["lon"] = lon
            df_forecast = pd.concat([df_forecast, df], ignore_index=True)
    # # store the data using the store_data function
    # store_data(df_forecast, path_forecast_data)
    return df_forecast

In [ ]:
print(get_forecast_data().head())

### Stable version of the API call before save to csv and create DF

In [ ]:
# Function to call the Openweather API one city
"""
Function to call the call_openweather_api(urm),
get the data and store it in a dataframe and append to csv file
"""


def get_weather_forecast():

    data = call_open_weather_api(url_forecast)

    if data is not None:

        df = pd.json_normalize(data["list"])

        # Add the new data with a timestamp of the time of the request
        df["timestamp"] = pd.Timestamp.now()

        # Convert the weather column to a single value
        df["weather"] = df["weather"].apply(lambda x: x[0]["id"])

        # define the columns to keep and rename them
        df = df[
            [
                "timestamp",
                "dt_txt",
                "weather",
                "main.temp",
                "main.humidity",
                "wind.speed",
                "clouds.all",
                "main.grnd_level",
                "rain.3h",
            ]
        ]
        df.columns = [
            "timestamp",
            "date",  # change to date_time_utc for the final version
            "weather_score",
            "temperature",
            "humidity",
            "windSpeed",
            "cloudCover",
            "pressure",
            "rain",
        ]
        # add the coord.lat - coord.lon from data['city']
        df["lat"] = data["city"]["coord"]["lat"]
        df["lon"] = data["city"]["coord"]["lon"]

        # replace "lat" and "lon" with the actual values
        df["lat"] = lat
        df["lon"] = lon

        # if empty space in the rain column, replace with 0
        df["rain"] = df["rain"].fillna(0)

        # insert the data into the csv file
        if pathlib.Path(path_forecast_csv).exists():
            df.to_csv(path_forecast_csv, mode="a", header=False, index=False)
        else:
            df.to_csv(path_forecast_csv, mode="w", header=True, index=False)

        return df

    else:

        return None

    # to improve: handle errors and exceptions
    # connection error, file not found, etc

# Function to automate inoput data every x minutes
Recurrent function to be automatised and executed with regularity to updated data regularly for the site
- time_api_call: time in minutes to parameter the regularity of the automatisation

In [ ]:
# Define a schedule to run the get_weather_forecast function every 15 minutes
def automate_forecast():
    # start time
    start_time_automate = time.time()
    schedule.every(up_time).minutes.do(get_weather_forecast)
    while True:
        schedule.run_pending()
        time.sleep(1)
        # end time
        end_time_automate = time.time()
        print("Execution time: ", end_time_automate - start_time_automate)

Function to use the 

### Area to call funtions

In [ ]:
# get_forecast()

# data_forecast = call_open_weather_api(url_forecast)

# data_forecast

# get_weather_forecast()

automate_forecast()

In [ ]:
# # From data create a data frame with 'temp'
# df_forecast_full = pd.json_normalize(data_forecast['list'])
# df_forecast_city = pd.json_normalize(data_forecast['city'])
# df_forecast_full.head()
# df_forecast_city.tail()
# df_forecast = df_forecast_full[['dt_txt','main.temp','main.humidity','wind.speed','clouds.all','main.grnd_level', 'rain.3h']]
# df_forecast.columns = ['date','temperature','humidity','windSpeed','cloudCover','pressure','rain']
# df_forecast.head()
# df_forecast_full['weather']

In [ ]:
# print the value of column weather first row
print(df["weather"][0])

# Verify that the data downlodesd is ok
- Function to verfy that there is not error or empty values in the data before to send to timeseries

# To solve
- each fucntion should have thier own url variable, to target the specific needs (ok)
- Function get_weather should be executed every 15 minutes, starting from 00:00 every day
    - possile solution, create and automation that start at 00:00 for the automation that call the api every 15 minutes **Cron schedule in zapdos**

## Standardisation

In [ ]:
# # Function to wrangle df
# def post_processing():
#     # Replace missing values with 0 in column: 'rain'
#     df = df.fillna({"rain": 0})

#     # Performed 10 aggregations grouped on column: 'date'
#     df = (
#         df.groupby(["date"])
#         .agg(
#             timestamp_last=("timestamp", "last"),
#             weather_score_mode=("weather_score", lambda s: s.value_counts().index[0]),
#             temperature_mean=("temperature", "mean"),
#             humidity_mean=("humidity", "mean"),
#             windSpeed_mean=("windSpeed", "mean"),
#             cloudCover_mean=("cloudCover", "mean"),
#             pressure_mode=("pressure", lambda s: s.value_counts().index[0]),
#             rain_mean=("rain", "mean"),
#             lat_mean=("lat", "mean"),
#             lon_mean=("lon", "mean"),
#         )
#         .reset_index()
#     )